In [1]:
# Cell 1: Setup
import os
import torch
from ultralytics import YOLO
import yaml
import glob

DATA_YAML = "/home/kakani/waymo_project/2_yolo_dataset/data.yaml"
print("✅ DATA_YAML:", DATA_YAML)
print("✅ YAML Exists:", os.path.exists(DATA_YAML))

# Display data.yaml contents
with open(DATA_YAML, 'r') as f:
    print("\n📄 data.yaml contents:")
    print(yaml.dump(yaml.safe_load(f), default_flow_style=False))


✅ DATA_YAML: /home/kakani/waymo_project/2_yolo_dataset/data.yaml
✅ YAML Exists: True

📄 data.yaml contents:
names:
  0: VEHICLE
  1: PEDESTRIAN
  2: SIGN
  3: CYCLIST
path: /home/kakani/waymo_project/2_yolo_dataset
train: images/train
val: images/val



In [2]:
# Cell 2: Check device
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Using device:", DEVICE)

if DEVICE == "cuda":
    print("✅ GPU name:", torch.cuda.get_device_name(0))

Torch: 2.5.1+cu121
CUDA available: True
CUDA device count: 1
✅ Using device: cuda
✅ GPU name: NVIDIA GeForce RTX 4060 Laptop GPU


In [3]:
# Cell 3: Verify dataset structure (excluding .ipynb_checkpoints)
YOLO_ROOT = "/home/kakani/waymo_project/2_yolo_dataset"
train_img_dir = os.path.join(YOLO_ROOT, "images/train")
train_lbl_dir = os.path.join(YOLO_ROOT, "labels/train")
val_img_dir = os.path.join(YOLO_ROOT, "images/val")
val_lbl_dir = os.path.join(YOLO_ROOT, "labels/val")

# Get only valid files (not directories, not hidden)
train_images = [f for f in os.listdir(train_img_dir) 
                if f.endswith(".jpg") and not f.startswith('.')]
train_labels = [f for f in os.listdir(train_lbl_dir) 
                if f.endswith(".txt") and not f.startswith('.')]
val_images = [f for f in os.listdir(val_img_dir) 
              if f.endswith(".jpg") and not f.startswith('.')]
val_labels = [f for f in os.listdir(val_lbl_dir) 
              if f.endswith(".txt") and not f.startswith('.')]

print("\n📊 DATASET VERIFICATION:")
print("-" * 40)
print(f"✅ Train images: {len(train_images)}")
print(f"✅ Train labels: {len(train_labels)}")
print(f"✅ Val images: {len(val_images)}")
print(f"✅ Val labels: {len(val_labels)}")

# Check total instances in training set
total_train_instances = 0
for lbl_file in train_labels:
    with open(os.path.join(train_lbl_dir, lbl_file)) as f:
        total_train_instances += len(f.readlines())
print(f"✅ Total training instances: {total_train_instances}")

# Check total instances in validation set
total_val_instances = 0
for lbl_file in val_labels:
    with open(os.path.join(val_lbl_dir, lbl_file)) as f:
        total_val_instances += len(f.readlines())
print(f"✅ Total validation instances: {total_val_instances}")

# Check a sample label
if train_labels:
    sample = sorted(train_labels)[0]
    print(f"\n✅ Sample label file: {sample}")
    with open(os.path.join(train_lbl_dir, sample)) as f:
        content = f.read()
        print(content[:200] if content else "(empty)")



📊 DATASET VERIFICATION:
----------------------------------------
✅ Train images: 30
✅ Train labels: 30
✅ Val images: 20
✅ Val labels: 20
✅ Total training instances: 187
✅ Total validation instances: 492

✅ Sample label file: train_frame_0000.txt
0 0.763829 0.384875 0.125331 0.085857
0 0.577642 0.419415 0.093094 0.121877
0 0.615471 0.405599 0.032237 0.074508
0 0.524022 0.380434 0.014803 0.030593
0 0.718763 0.471719 0.188161 0.099673


In [4]:
# Cell 4: Load model
model = YOLO("yolov8n.pt")
print("✅ YOLOv8n Loaded")

✅ YOLOv8n Loaded


In [5]:
# Cell 5: Train model
print("\n🚀 STARTING TRAINING...")
print("=" * 50)

train_results = model.train(
    data=DATA_YAML,
    epochs=200,
    imgsz=640,
    batch=8,
    device=DEVICE,
    project="/home/kakani/waymo_project/4_runs",
    name="waymo_yolo_train_fixed",
    verbose=True,
    patience=40,# Early stopping if no improvement
    augment=True,
    mosaic=1.0,
    mixup=0.2,
    save=True,
    plots=True,
    exist_ok=True  # Overwrite if exists
)

print("\n✅ Training complete!")


🚀 STARTING TRAINING...
New https://pypi.org/project/ultralytics/8.4.14 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.2 🚀 Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/kakani/waymo_project/2_yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8n.pt, momentum=0.93

In [6]:
# Cell 6: Validate model
print("\n🔍 VALIDATING MODEL...")
metrics = model.val(data=DATA_YAML, device=DEVICE)

print("\n📊 VALIDATION METRICS:")
print("-" * 40)
print(f"Precision: {metrics.box.map50:.4f}")
print(f"Recall: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

# Per-class metrics if available
if hasattr(metrics, 'ap_class_index') and len(metrics.ap_class_index) > 0:
    print("\n📊 PER-CLASS METRICS:")
    names = {0: 'VEHICLE', 1: 'PEDESTRIAN', 2: 'SIGN', 3: 'CYCLIST'}
    for i, class_id in enumerate(metrics.ap_class_index):
        if i < len(metrics.box.maps):
            print(f"{names[class_id]}: mAP50-95 = {metrics.box.maps[i]:.4f}")



🔍 VALIDATING MODEL...
Ultralytics 8.4.2 🚀 Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 73 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1864.8±537.1 MB/s, size: 461.9 KB)
val: Scanning /home/kakani/waymo_project/2_yolo_dataset/labels/val.cache... 20 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 20/20 3.1Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 1.8s/it 3.6s
                   all         20        492      0.629     0.0595     0.0523     0.0262
               VEHICLE         20        395      0.257      0.119      0.105     0.0524
            PEDESTRIAN         20         97          1          0          0          0
Speed: 7.6ms preprocess, 110.2ms inference, 0.0ms loss, 38.5ms postprocess per image
Results saved to /home/kakani/waymo_project/3_notebooks/runs/detect/val11

📊 V

In [7]:
# Cell 7: Test on a single image
print("\n🧪 TESTING ON SINGLE IMAGE...")
if train_images:
    test_img = os.path.join(train_img_dir, train_images[0])
    print("✅ Testing on:", os.path.basename(test_img))

    pred = model.predict(
        source=test_img,
        conf=0.25,
        save=True,
        project="/home/kakani/waymo_project/4_runs",
        name="predictions_fixed",
        exist_ok=True
    )
    print("✅ Prediction complete!")


🧪 TESTING ON SINGLE IMAGE...
✅ Testing on: train_frame_0025.jpg

image 1/1 /home/kakani/waymo_project/2_yolo_dataset/images/train/train_frame_0025.jpg: 448x640 10 VEHICLEs, 67.5ms
Speed: 9.7ms preprocess, 67.5ms inference, 3.1ms postprocess per image at shape (1, 3, 448, 640)
Results saved to /home/kakani/waymo_project/4_runs/predictions_fixed
✅ Prediction complete!


In [8]:
# Cell 8: Test on all validation images
print("\n🧪 TESTING ON ALL VALIDATION IMAGES...")
VAL_IMG_DIR = "/home/kakani/waymo_project/2_yolo_dataset/images/val"

if val_images:
    results = model.predict(
        source=VAL_IMG_DIR,
        conf=0.25,
        save=True,
        project="/home/kakani/waymo_project/4_runs",
        name="val_predictions_fixed",
        exist_ok=True
    )
    print(f"✅ Predictions done on {len(val_images)} VAL images!")



🧪 TESTING ON ALL VALIDATION IMAGES...

image 1/20 /home/kakani/waymo_project/2_yolo_dataset/images/val/val_frame_0000.jpg: 448x640 12 VEHICLEs, 18.5ms
image 2/20 /home/kakani/waymo_project/2_yolo_dataset/images/val/val_frame_0001.jpg: 448x640 25 VEHICLEs, 18.7ms
image 3/20 /home/kakani/waymo_project/2_yolo_dataset/images/val/val_frame_0002.jpg: 448x640 22 VEHICLEs, 18.5ms
image 4/20 /home/kakani/waymo_project/2_yolo_dataset/images/val/val_frame_0003.jpg: 448x640 29 VEHICLEs, 20.1ms
image 5/20 /home/kakani/waymo_project/2_yolo_dataset/images/val/val_frame_0004.jpg: 448x640 21 VEHICLEs, 18.2ms
image 6/20 /home/kakani/waymo_project/2_yolo_dataset/images/val/val_frame_0005.jpg: 448x640 23 VEHICLEs, 18.3ms
image 7/20 /home/kakani/waymo_project/2_yolo_dataset/images/val/val_frame_0006.jpg: 448x640 16 VEHICLEs, 18.1ms
image 8/20 /home/kakani/waymo_project/2_yolo_dataset/images/val/val_frame_0007.jpg: 448x640 25 VEHICLEs, 18.3ms
image 9/20 /home/kakani/waymo_project/2_yolo_dataset/images/val/

In [9]:
# Cell 9: Check prediction results
PRED_DIR = "/home/kakani/waymo_project/4_runs/val_predictions_fixed"
print("\n📁 Prediction folder exists:", os.path.exists(PRED_DIR))

if os.path.exists(PRED_DIR):
    pred_images = [f for f in os.listdir(PRED_DIR) 
                   if f.endswith(".jpg") or f.endswith(".png")]
    print(f"✅ Predicted images: {len(pred_images)}")
    print("Sample predictions:", pred_images[:5])



📁 Prediction folder exists: True
✅ Predicted images: 20
Sample predictions: ['val_frame_0008.jpg', 'val_frame_0015.jpg', 'val_frame_0002.jpg', 'val_frame_0018.jpg', 'val_frame_0005.jpg']


In [10]:
# Cell 10: Visualize some predictions
import matplotlib.pyplot as plt
import cv2

if os.path.exists(PRED_DIR):
    pred_images = sorted([f for f in os.listdir(PRED_DIR) 
                         if f.endswith(".jpg") or f.endswith(".png")])
    
    for f in pred_images[:3]:  # Show first 3 predictions
        img_path = os.path.join(PRED_DIR, f)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(12,6))
        plt.imshow(img)
        plt.title(f"Prediction: {f}")
        plt.axis("off")
        plt.show()



<Figure size 1200x600 with 1 Axes>

<Figure size 1200x600 with 1 Axes>

<Figure size 1200x600 with 1 Axes>

In [11]:

# Cell 11: Save model path for future use
BEST_MODEL_PATH = "/home/kakani/waymo_project/4_runs/waymo_yolo_train_fixed/weights/best.pt"
print(f"\n💾 Best model saved at: {BEST_MODEL_PATH}")
print("✅ Model path exists:", os.path.exists(BEST_MODEL_PATH))


💾 Best model saved at: /home/kakani/waymo_project/4_runs/waymo_yolo_train_fixed/weights/best.pt
✅ Model path exists: True
